# Backtester Test

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.path.dirname('__file__'), '..')))

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from backtester.engine import BacktestEngine, Performance, PerfStats
from backtester.signals import TSMOM, VolatilityScale, as_freq

## Define Tickers

In [ ]:
TICKERS = {
    "Equity": ["ES=F", "NQ=F", "YM=F", "RTY=F"],
    "Bond": ["ZB=F", "ZN=F", "ZF=F", "ZT=F"],
    "FX": ["6E=F", "6J=F", "6B=F", "6A=F", "6C=F", "6S=F"],
    "Commodity": ["CL=F", "GC=F", "SI=F", "HG=F", "NG=F", "RB=F", "HO=F", "PA=F", "PL=F"],
}
all_tickers = [ticker for asset_class in TICKERS.values() for ticker in asset_class]

## Run Backtest

In [ ]:
engine = BacktestEngine(
    strategy="DELTA_ONE",
    instrument=all_tickers,
    signal=[
        VolatilityScale(target_volatility=0.4, ann_factor=261, com=60, signal_cap=0.95),
        TSMOM(lookback=252, shift=2)
    ],
    weighting="EQUAL_WEIGHT",
)
engine.run()

## Equity Curve

In [ ]:
equity_curve = engine.calculate_equity_curve()
equity_curve.plot(figsize=(10, 6), title='Backtester Equity Curve', logy=True)

## Performance Summary

In [ ]:
performance = Performance(equity_curve)
performance.summary()

## Sharpe Ratios by Instrument

In [ ]:
instrument_returns = engine.get_return_by_instrument()
performance_by_instrument = Performance(instrument_returns, return_type='diff')
sharpe_ratios = performance_by_instrument.summary().loc[PerfStats.SHARPE.value]
sharpe_ratios.plot(kind='bar', figsize=(14, 4), title='Sharpe Ratios by Instrument')